In [96]:
import numpy as np
from numpy import random
import math

In [97]:
customer_coords = np.loadtxt('customers_40.data')

In [98]:
def data_format(coords):
    data = {}
    idx = 1

    for i in coords:        
        data[idx] = (i[0], i[1])
        idx += 1

    return data

In [99]:
data = data_format(customer_coords)
#data

In [100]:
def calculate_route_cost(loc1, loc2):
     return math.sqrt((data[loc1][0] - data[loc2][0])**2 + (data[loc1][1] - data[loc2][1])**2)

In [101]:
trail_data = {}
ant_data = {}

In [102]:
def init_ants(coords_data, ants_count, ant_data, destination_node):
    init_nodes = random.choice(np.array([i for i in coords_data.keys() if i != destination_node]), ants_count, replace=False)
    
    i = 1

    for node in init_nodes:
        ant_data[i] = {'start_node': node, 'destination_node': destination_node, 'paths': []}
        i += 1

    return ant_data

In [103]:
init_ants(data, 9, ant_data, 10)

{1: {'start_node': 46, 'destination_node': 10, 'paths': []},
 2: {'start_node': 2, 'destination_node': 10, 'paths': []},
 3: {'start_node': 22, 'destination_node': 10, 'paths': []},
 4: {'start_node': 48, 'destination_node': 10, 'paths': []},
 5: {'start_node': 15, 'destination_node': 10, 'paths': []},
 6: {'start_node': 57, 'destination_node': 10, 'paths': []},
 7: {'start_node': 43, 'destination_node': 10, 'paths': []},
 8: {'start_node': 55, 'destination_node': 10, 'paths': []},
 9: {'start_node': 21, 'destination_node': 10, 'paths': []}}

In [104]:
def init_pheromone(coords_data, trail_data):
    for data_row in coords_data.keys():
        trail_row = []
        for data_col in coords_data.keys():
            if data_row == data_col:
                trail_row.append(0)
            else:
                if data_col > data_row:
                    trail_row.append(random.uniform(0, 1))
                else:
                    trail_row.append(trail_data[data_col][data_row - 1])
        trail_data[data_row] = trail_row

    return trail_data

In [105]:
init_pheromone(data, trail_data)
print(trail_data)

{1: [0, 0.5009848503483243, 0.6487959852999678, 0.33296027656525173, 0.5657354231309906, 0.546526643028594, 0.3684108076118833, 0.13513172925689287, 0.8563345619231708, 0.3974098854371574, 0.18112830377904066, 0.2835863803604921, 0.3231240991971761, 0.17786769125609425, 0.5547340688102514, 0.6309677123013722, 0.16035947725377808, 0.042953549664181745, 0.48006811980922315, 0.14651149078812797, 0.8503294413602893, 0.5544942786648335, 0.8496937047253095, 0.2646350290127637, 0.7620059150609741, 0.6349018276616635, 0.45066783666602006, 0.44069119732265116, 0.7503587512118558, 0.08170516737996569, 0.2955259452042135, 0.9220534281623292, 0.5381288067997597, 0.8127680210035563, 0.3613136290212108, 0.08799454391905226, 0.8383736288176835, 0.7854830795875182, 0.9432788449522107, 0.64870815623949, 0.31627116403395106, 0.7922105234712751, 0.5192812870208844, 0.8697823376111233, 0.401950190418631, 0.8759969540709254, 0.5035110538609721, 0.7414664633349489, 0.6814442465201647, 0.2990899702628561, 0.

In [106]:
def calculate_route_costs(coords_data):
    distance_data = []

    for data_row in coords_data.keys():
        distance_row = []
        for data_col in coords_data.keys():
            if data_row != data_col:
                distance_row.append(calculate_route_cost(data_row, data_col))
            else:
                distance_row.append(0)
        distance_data.append(distance_row)

    return distance_data

In [107]:
distance_data = calculate_route_costs(data)
#distance_data

In [108]:
def traverse_nodes(ant_data, destination_node, coords_data, distance_data, trail_data, alpha = 0.6):
    for ant in ant_data.keys():
        current_node = ant_data[ant]['start_node']
        ant_data[ant]['paths'].append({'path': [current_node], 'cost': 0})
        # if the current path is not destination node
        while current_node != ant_data[ant]['destination_node']:
            # loop through all nodes and calculate sum of probabilities
            sum_of_probabilities = 0
            for node in coords_data.keys():
                if node != current_node:
                    sum_of_probabilities += (trail_data[current_node][node - 1] * alpha) + \
                    ((1 - alpha) * (1 / distance_data[current_node - 1][node - 1]))
            path_probabilities = [(None, 0)]
            # loop through all nodes and calculate probabilities
            for node in coords_data.keys():
                # dont calculate probability of visiting itself
                if node != current_node:
                    probability = ((trail_data[current_node][node - 1] * alpha) + \
                    ((1 - alpha) * (1 / distance_data[current_node - 1][node - 1]))) \
                    / sum_of_probabilities
                    # do a cummulative probability that will be used to determine next path based
                    path_probabilities.append((node, path_probabilities[-1][1] + probability))
            rand_selector = random.uniform(0, 1)
            for idx, path in enumerate(path_probabilities):
                if rand_selector > path_probabilities[idx][1] and rand_selector < path_probabilities[idx + 1][1]:
                    current_node = path_probabilities[idx + 1][0]
                    ant_data[ant]['paths'][-1]['path'].append(current_node)
                    break
                 
    return           

In [109]:
traverse_nodes(ant_data, 10, data, distance_data, trail_data)
print(ant_data)

{1: {'start_node': 46, 'destination_node': 10, 'paths': [{'path': [46, 13, 57, 40, 34, 5, 33, 4, 57, 12, 43, 10], 'cost': 0}]}, 2: {'start_node': 2, 'destination_node': 10, 'paths': [{'path': [2, 58, 59, 14, 18, 9, 56, 48, 41, 47, 60, 50, 34, 49, 51, 41, 51, 9, 32, 52, 33, 38, 14, 41, 48, 30, 40, 36, 59, 40, 6, 40, 3, 37, 1, 46, 44, 17, 54, 26, 23, 57, 47, 10], 'cost': 0}]}, 3: {'start_node': 22, 'destination_node': 10, 'paths': [{'path': [22, 9, 26, 12, 56, 22, 48, 56, 7, 35, 29, 5, 10], 'cost': 0}]}, 4: {'start_node': 48, 'destination_node': 10, 'paths': [{'path': [48, 52, 51, 14, 60, 45, 19, 51, 25, 3, 41, 24, 13, 34, 50, 14, 20, 32, 56, 9, 51, 54, 23, 25, 23, 45, 14, 6, 14, 43, 10], 'cost': 0}]}, 5: {'start_node': 15, 'destination_node': 10, 'paths': [{'path': [15, 16, 49, 48, 22, 46, 47, 49, 18, 27, 52, 56, 57, 5, 10], 'cost': 0}]}, 6: {'start_node': 57, 'destination_node': 10, 'paths': [{'path': [57, 34, 9, 39, 19, 38, 48, 29, 57, 25, 15, 33, 13, 53, 20, 43, 4, 23, 21, 56, 52, 6,

In [110]:
def remove_path_loops(ant_data):
    for ant, data in ant_data.items():
        latest_path = data['paths'][-1]['path']
        modified = True

        while modified:
            modified = False
            for idx in range(len(latest_path)):
                last_idx = len(latest_path) - 1
                while idx < last_idx:
                    if latest_path[idx] == latest_path[last_idx]:
                        latest_path = latest_path[:idx] + latest_path[last_idx:]
                        modified = True
                        break
                    last_idx -= 1
                if modified:
                    break 
        ant_data[ant]['paths'][-1]['path'] = latest_path

    return

In [111]:
#remove_path_loops(ant_data)

In [112]:
#ant_data

In [113]:
def calculate_path_total_cost(ant_data, distance_data):
    for ant, data in ant_data.items():
        latest_path = data['paths'][-1]['path']
        path_cost = data['paths'][-1]['cost']
        if len(latest_path) > 1:
            for idx in range(len(latest_path) - 1):
                path_cost += distance_data[latest_path[idx] - 1][latest_path[idx + 1] - 1]
            ant_data[ant]['paths'][-1]['cost'] = path_cost    

    return

In [114]:
#calculate_path_total_cost(ant_data, distance_data)

In [115]:
#ant_data

In [116]:
def negative_feedback(evaporation_rate, trail_data):
    for node, pheromone_data in trail_data.items():
        for idx, pheromone in enumerate(pheromone_data):
            trail_data[node][idx] = (1 - evaporation_rate) * pheromone

    return

In [117]:
#print(trail_data)

In [118]:
#negative_feedback(0.45, trail_data)

In [119]:
#print(trail_data)

In [120]:
def positive_feedback(Q, ant_data, trail_data):
    for ant, data in ant_data.items():
        latest_path = data['paths'][-1]['path']

        if len(latest_path) > 1:
            for idx in range(len(latest_path) - 1):
                # increase pheromone for path in both directions [a -> b] == [b -> a]
                trail_data[latest_path[idx]][latest_path[idx + 1] - 1] += Q / distance_data[latest_path[idx] - 1][latest_path[idx + 1] - 1]
                trail_data[latest_path[idx + 1]][latest_path[idx] - 1] += Q / distance_data[latest_path[idx] - 1][latest_path[idx + 1] - 1]

    return

In [121]:
#positive_feedback(1, ant_data, trail_data)

In [122]:
#print(trail_data)

In [123]:
def ant_colony_optimization(coords, Q=1, evaporate=0.45, dest_node=60, ants_count=30, max_iterations=20):
    data = data_format(coords)
    trail_data = {}
    ant_data = {}
    init_ants(data, ants_count, ant_data, dest_node)
    init_pheromone(data, trail_data)
    distance_data = calculate_route_costs(data)
    for i in range(max_iterations):
        traverse_nodes(ant_data, dest_node, data, distance_data, trail_data)
        remove_path_loops(ant_data)
        calculate_path_total_cost(ant_data, distance_data)
        negative_feedback(evaporate, trail_data)
        positive_feedback(Q, ant_data, trail_data)
    for ant, data in ant_data.items():
        print("ant {}: ({})".format(ant, data))
        print("--------------------------------------------------------")

In [124]:
ant_colony_optimization(customer_coords)

ant 1: ({'start_node': 45, 'destination_node': 60, 'paths': [{'path': [45, 3, 22, 36, 35, 47, 56, 15, 50, 60], 'cost': 339.8995865155387}, {'path': [45, 54, 15, 29, 18, 53, 49, 24, 28, 60], 'cost': 427.62565126405497}, {'path': [45, 59, 24, 9, 23, 60], 'cost': 286.24711786568764}, {'path': [45, 56, 7, 20, 49, 59, 12, 58, 28, 60], 'cost': 409.9760446567949}, {'path': [45, 55, 18, 16, 7, 26, 24, 28, 50, 60], 'cost': 393.24923419228685}, {'path': [45, 44, 10, 60], 'cost': 102.30321824811254}, {'path': [45, 22, 51, 41, 52, 35, 57, 39, 60], 'cost': 333.13128715462875}, {'path': [45, 24, 11, 27, 28, 39, 30, 60], 'cost': 289.9229445180585}, {'path': [45, 47, 34, 40, 15, 48, 2, 30, 60], 'cost': 191.09435649498394}, {'path': [45, 55, 34, 47, 30, 60], 'cost': 93.21712504561337}, {'path': [45, 55, 39, 43, 21, 53, 19, 56, 60], 'cost': 266.4621060623159}, {'path': [45, 35, 60], 'cost': 89.97741174225305}, {'path': [45, 55, 37, 24, 36, 54, 35, 27, 51, 21, 26, 28, 4, 60], 'cost': 317.57194324512363},